<a href="https://colab.research.google.com/github/Buuuuli/AIPI540_recommendation/blob/main/DL_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT, force_remount=False)

Mounted at /content/drive


In [39]:
%cd /content/drive/"My Drive"/AIPI540_recom/data

/content/drive/My Drive/AIPI540_recom/data


In [38]:
import sys
import os
import torch
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
import pickle as pkl
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print("System version: {}".format(sys.version))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
PyTorch version: 1.10.0+cu111
Cornac version: 1.14.2


In [7]:
df = pd.read_pickle('final_dataframe.pkl')

In [8]:
df_new = df.sample(20000)

In [9]:
data_encoded = df_new.copy()
for col in ['business_id','user_id']:
    data_encoded[col] = data_encoded[col].astype('category') # Convert to category type
    data_encoded[col] = data_encoded[col].cat.codes # Convert to numerical code
data_encoded.head()

,business_id,name,address,city,state,postal_code,stars,review_count,categories,user_id,review_stars
232861,3539,Maple Street Biscuit Company - Seminole City,"7756 113th St N, Ste E",Seminole,FL,33772,4.0,151,"Breakfast & Brunch, Southern, Restaurants, Com...",3976,2
508775,3409,The Winey Wench,"27001 US Hwy 19 N, Ste 1072",Clearwater,FL,33761,5.0,6,"Food, Wineries, Arts & Entertainment, Arts & C...",16286,4
20839,4545,Bambino Pizza,2028 Bayshore Blvd,Dunedin,FL,34698,3.0,29,"Pizza, Italian, Food Delivery Services, Food, ...",75,3
329248,686,Marlene's Original Breakfast Sandwich,7206 N Dale Mabry Hwy,Tampa,FL,33614,5.0,127,"Food, Food Trucks, Sandwiches, Restaurants, Br...",13019,4
339150,951,Revelations Cafe,17808 N Dale Mabry Hwy,Lutz,FL,33548,4.5,111,"Juice Bars & Smoothies, Vegetarian, Vegan, Caf...",13656,5


In [10]:
data_encoded_only = data_encoded[['business_id','user_id','review_stars']]

In [11]:
data_encoded_only = data_encoded_only.reset_index(drop=True)

In [12]:
data_encoded_only

,business_id,user_id,review_stars
0,3539,3976,2
1,3409,16286,4
2,4545,75,3
3,686,13019,4
4,951,13656,5
...,...,...,...
19995,4475,5998,3
19996,3926,12689,5
19997,1074,12752,4
19998,3482,1660,4


In [13]:
data_encoded_only.rename(columns = {'business_id':'itemID', 'user_id':'userID','review_stars':'rating'}, inplace = True)

In [14]:
train, test = python_random_split(data_encoded_only, 0.75)

In [15]:
# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 5
ENCODER_DIMS = [300]
ACT_FUNC = "tanh"
LIKELIHOOD = "gaus"
NUM_EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [16]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 4187
Number of items: 12781


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 15 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [17]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 1098.1163 seconds for training.


In [18]:
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 58.4245 seconds for prediction.


In [40]:
#with open('./all_prediction.pkl','wb') as fp:
  #pkl.dump(all_predictions,fp)

In [24]:
all_predictions

,userID,itemID,prediction
15000,1934,5758,0.001560
15001,1934,9318,0.001701
15002,1934,4037,0.001643
15003,1934,3486,0.001544
15004,1934,14606,0.001544
...,...,...,...
53526090,2749,8187,0.001467
53526091,2749,11321,0.001789
53526092,2749,15540,0.001678
53526093,2749,12109,0.001212


In [30]:
t = all_predictions.loc[(all_predictions['userID'] == 1934)]

In [36]:
m = t.sort_values(by=['prediction'], ascending=False)

In [47]:
j = m.iloc[:3]['itemID'].tolist()

In [48]:
j

[1622, 14059, 15889]

In [54]:
data_encoded.loc[data_encoded['business_id'].isin(j)]

,business_id,name,address,city,state,postal_code,stars,review_count,categories,user_id,review_stars
519420,1622,Bogie's Food and Flicks,"33 N Garden Ave, Ste 140",Clearwater,FL,33755,5.0,9,"Restaurants, Breakfast & Brunch, Salad, Sandwi...",12954,4


In [28]:
all_predictions.loc[all_predictions.userID == 2749, 'userID'].count()

12780

In [29]:
all_predictions.loc[all_predictions.userID == 2749, 'itemID'].count()

12780

In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.000083
NDCG:	0.000239
Precision@K:	0.000083
Recall@K:	0.000827


In [ ]:
%cd /content/drive/"My Drive"/AIPI540_recom/data

/content/drive/My Drive/AIPI540_recom/data


In [ ]:

filename = 'bivae_model.pt'

# Save the entire model
torch.save(bivae, filename)

In [ ]:
model = torch.load(filename)

c = predict_ranking(model, train, usercol='userID', itemcol='itemID', remove_seen=True)
